In [44]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import escalateclient
import importlib
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
# server_url = 'https://escalate.cs.haverford.edu/test_server'
username = 'vshekar'
password = 'copperhead123'
client = escalateclient.ESCALATEClient(server_url, username, password)

In [46]:
neilson_lab_response = client.get_or_create(endpoint='actor', data={'description': 'Neilson Lab', 'person__isnull': True, 'systemtool__isnull': True})[0]

{'description': 'Neilson Lab', 'person__isnull': True, 'systemtool__isnull': True}
GET: OK. Found 1 results


In [47]:
reagent_templates = {
            "Reagent 1 - Nitride Precursors": {"chemical_types": ["nitride"]},
            "Reagent 2 - Oxide Precursors": {"chemical_types": ["oxide"]},
            "Reagent 3 - Fluxes": {"chemical_types": ["flux"]}
        }

reagent_template_responses, rmt_responses = client.create_reagent_templates(data=reagent_templates)

{'description': 'Reagent 1 - Nitride Precursors'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'nitride'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Reagent 1 - Nitride Precursors: nitride', 'reagent_template': 'ae7e3660-cd01-43cc-b907-6c05fce855f4', 'material_type': '9cd2ec88-172e-4873-bd27-18c1ca7f2fcf'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Reagent 2 - Oxide Precursors'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'oxide'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Reagent 2 - Oxide Precursors: oxide', 'reagent_template': '737abf04-a1de-4f90-99cc-d5eea0f4cbc4', 'material_type': '915ee791-c863-464b-97f2-8da957eba893'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Reagent 3 - Fluxes'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'flux'}

In [48]:
zero_g_value = {"value": 0, "unit": "g", "type": "num"}
zero_g_data = {"description": "Zero g", "nominal_value": zero_g_value, "actual_value": zero_g_value,}
zero_g_response = client.get_or_create(endpoint='default-values', data=zero_g_data)[0]

nitride = client.get(endpoint='reagent-template', 
                     resource_id=reagent_template_responses['Reagent 1 - Nitride Precursors']['uuid'], 
                     related_endpoint='reagent-material-template')[0]
oxide = client.get(endpoint='reagent-template', 
                     resource_id=reagent_template_responses['Reagent 2 - Oxide Precursors']['uuid'], 
                     related_endpoint='reagent-material-template')[0]
flux = client.get(endpoint='reagent-template', 
                     resource_id=reagent_template_responses['Reagent 3 - Fluxes']['uuid'], 
                     related_endpoint='reagent-material-template')[0]


data = {"description": "amount",
        "default_value": zero_g_response['url']}
prop_amount = client.get_or_create('property-template', data=data)[0]
nitride['properties'].append(prop_amount['url'])
client.patch('reagent-material-template', nitride['uuid'], {'properties': nitride['properties']})

oxide['properties'].append(prop_amount['url'])
client.patch('reagent-material-template', oxide['uuid'], {'properties': oxide['properties']})

flux['properties'].append(prop_amount['url'])
client.patch('reagent-material-template', flux['uuid'], {'properties': flux['properties']})


{'description': 'Zero g'}
GET: OK. Found 1 results
{}
GET: OK. Found 1 results
{}
GET: OK. Found 1 results
{}
GET: OK. Found 1 results
{'description': 'amount', 'default_value': '994c557c-a74d-4d3c-8591-48274cb25faf'}
GET: OK. Found 1 results
Status 200: OK
Status 200: OK
Status 200: OK


{'url': 'http://localhost:8000/api/reagent-material-template/7b6c9be0-244b-4273-b080-b936c11053a7/',
 'uuid': '7b6c9be0-244b-4273-b080-b936c11053a7',
 'edocs': [],
 'tags': [],
 'notes': [],
 'add_date': '2022-04-20T22:31:31.695672',
 'mod_date': '2022-04-20T22:31:36.158357',
 'description': 'Reagent 3 - Fluxes: flux',
 'status': None,
 'reagent_template': 'http://localhost:8000/api/reagent-template/32aa0209-4626-4fe8-9941-613a74dc9ca0/',
 'material_type': 'http://localhost:8000/api/material-type/fd3e16a4-3a60-40a6-ad4b-868144c5229d/',
 'properties': ['http://localhost:8000/api/property-template/7d33e094-1b6e-464a-a260-3c81f8d6019d/']}

In [49]:
action_parameter_def = {
            "bring_to_temperature": [("temperature", {"type": "num", "unit": "degC", "value": 0.0})],
            "dwell": [("duration", {"type": "num", "unit": "seconds", "value": 0.0})],
            "cool_to_temperature": [("temperature", {"type": "num", "unit": "degC", "value": 0.0})],
        }
# Convenience function to add action defs and their parameter definitions
action_defs, parameter_defs = client.create_action_parameters(data=action_parameter_def)


{'description': 'temperature'}
GET: OK. Found 1 results
{'description': 'bring_to_temperature', 'parameter_def': ['http://localhost:8000/api/parameter-def/1236f3f6-820d-4539-af40-a348114d227d/']}
GET: OK. Found 0 results
POST: OK, returning new resource dict
[('temperature', {'type': 'num', 'unit': 'degC', 'value': 0.0})]
{'description': 'duration'}
GET: OK. Found 1 results
{'description': 'dwell', 'parameter_def': ['http://localhost:8000/api/parameter-def/1d572299-e12c-4b98-8743-dc6e1c568d59/']}
GET: OK. Found 0 results
POST: OK, returning new resource dict
[('duration', {'type': 'num', 'unit': 'seconds', 'value': 0.0})]
{'description': 'temperature'}
GET: OK. Found 1 results
{'description': 'cool_to_temperature', 'parameter_def': ['http://localhost:8000/api/parameter-def/1236f3f6-820d-4539-af40-a348114d227d/']}
GET: OK. Found 0 results
POST: OK, returning new resource dict
[('temperature', {'type': 'num', 'unit': 'degC', 'value': 0.0})]


In [50]:
action_defs['dispense'] = client.get('action-def', data={'description':'dispense'})[0]

{'description': 'dispense'}
GET: OK. Found 1 results


In [51]:
# Create Vessel templates
vessel_templates = {}
for vessel_template_description in ['Nitride Vessel', 'Oxide Vessel', 'Flux Vessel', 'Outcome vessel']:
    vessel_templates[vessel_template_description] = client.get_or_create('vessel-template', data={'description': vessel_template_description, 'outcome_vessel': False})[0]

{'description': 'Nitride Vessel'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Oxide Vessel'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Flux Vessel'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Outcome vessel'}
GET: OK. Found 1 results


In [52]:
outcome_response = client.get_or_create('outcome-template', data={'description': 'Neilson Outcome'})[0]


{'description': 'Neilson Outcome'}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [56]:
# Add vessel templates to Experiment template
vt_urls = [vt['url'] for vt in vessel_templates.values()]
rt_urls = [rt['url'] for rt in reagent_template_responses.values()]
ot_urls = [outcome_response['url']]
experiment_template_data = {'description': 'Neilson Experiment', 
                            'ref_id': 'workflow_1', 
                            'lab': neilson_lab_response['url'],
                            'vessel_templates': vt_urls,
                            'reagent_templates': rt_urls,
                            'outcome_templates': ot_urls
                            }
template_response = client.get_or_create(endpoint='experiment-template', data=experiment_template_data)[0]



{'description': 'Neilson Experiment', 'ref_id': 'workflow_1', 'lab': '73f43230-72f9-4263-adcf-17c38ea29dac', 'vessel_templates': ['http://localhost:8000/api/vessel-template/27327a1e-ea21-41b5-a3b5-ba001f7d1b42/', 'http://localhost:8000/api/vessel-template/c71e0a84-ddf0-45d6-87cb-6475e9ad71b6/', 'http://localhost:8000/api/vessel-template/73b6e4bd-28bd-4f17-a6e4-c5b9ff171f07/', 'http://localhost:8000/api/vessel-template/4f80aeeb-ee0d-43bf-8451-3b696b1c7946/'], 'reagent_templates': ['http://localhost:8000/api/reagent-template/ae7e3660-cd01-43cc-b907-6c05fce855f4/', 'http://localhost:8000/api/reagent-template/737abf04-a1de-4f90-99cc-d5eea0f4cbc4/', 'http://localhost:8000/api/reagent-template/32aa0209-4626-4fe8-9941-613a74dc9ca0/'], 'outcome_templates': ['http://localhost:8000/api/outcome-template/bb678a09-5ee4-4125-a411-3081577d91b7/']}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [57]:
rt_urls

['http://localhost:8000/api/reagent-template/ae7e3660-cd01-43cc-b907-6c05fce855f4/',
 'http://localhost:8000/api/reagent-template/737abf04-a1de-4f90-99cc-d5eea0f4cbc4/',
 'http://localhost:8000/api/reagent-template/32aa0209-4626-4fe8-9941-613a74dc9ca0/']

In [58]:
# Create action templates
dispense_nitrides_data = {
    "description": "Dispense Nitrides",
    "source_vessel_decomposable": False,
    "dest_vessel_decomposable": False,
    "experiment_template": template_response['url'],
    "action_def": action_defs['dispense']['url'],
    "source_vessel_template": vessel_templates['Nitride Vessel']['url'],
    "dest_vessel_template": vessel_templates['Outcome vessel']['url'],
    "parent": []
}
dispense_nitrides_data = client.get_or_create('action-template', data=dispense_nitrides_data)[0]


{'description': 'Dispense Nitrides', 'experiment_template': '9662fef4-7fb0-4a4c-b168-9afb89314693', 'action_def': 'ffdab19b-b8cf-47a1-822b-b396c20f7b02', 'source_vessel_template': '27327a1e-ea21-41b5-a3b5-ba001f7d1b42', 'dest_vessel_template': '4f80aeeb-ee0d-43bf-8451-3b696b1c7946'}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [59]:
dispense_oxides_data = {
    "description": "Dispense Oxides",
    "source_vessel_decomposable": False,
    "dest_vessel_decomposable": False,
    "experiment_template": template_response['url'],
    "action_def": action_defs['dispense']['url'],
    "source_vessel_template": vessel_templates['Oxide Vessel']['url'],
    "dest_vessel_template": vessel_templates['Outcome vessel']['url'],
    "parent": [dispense_nitrides_data['url']]
}
dispense_oxides_data = client.get_or_create('action-template', data=dispense_oxides_data)[0]

{'description': 'Dispense Oxides', 'experiment_template': '9662fef4-7fb0-4a4c-b168-9afb89314693', 'action_def': 'ffdab19b-b8cf-47a1-822b-b396c20f7b02', 'source_vessel_template': 'c71e0a84-ddf0-45d6-87cb-6475e9ad71b6', 'dest_vessel_template': '4f80aeeb-ee0d-43bf-8451-3b696b1c7946', 'parent': ['http://localhost:8000/api/action-template/d1343e47-9aa0-4e9f-bbc3-abfa03a3218d/']}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [60]:
dispense_flux_data = {
    "description": "Dispense Flux",
    "source_vessel_decomposable": False,
    "dest_vessel_decomposable": False,
    "experiment_template": template_response['url'],
    "action_def": action_defs['dispense']['url'],
    "source_vessel_template": vessel_templates['Flux Vessel']['url'],
    "dest_vessel_template": vessel_templates['Outcome vessel']['url'],
    "parent": [dispense_oxides_data['url']]
}
dispense_flux_data = client.get_or_create('action-template', data=dispense_flux_data)[0]

{'description': 'Dispense Flux', 'experiment_template': '9662fef4-7fb0-4a4c-b168-9afb89314693', 'action_def': 'ffdab19b-b8cf-47a1-822b-b396c20f7b02', 'source_vessel_template': '73b6e4bd-28bd-4f17-a6e4-c5b9ff171f07', 'dest_vessel_template': '4f80aeeb-ee0d-43bf-8451-3b696b1c7946', 'parent': ['http://localhost:8000/api/action-template/aaddf54f-2c44-4677-b0f4-9f802b4caaf9/']}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [61]:
action_defs.keys()

dict_keys(['bring_to_temperature', 'dwell', 'cool_to_temperature', 'dispense'])

In [62]:
heat_to_t_data = {
    "description": "Heat to T",
    "source_vessel_decomposable": False,
    "dest_vessel_decomposable": False,
    "experiment_template": template_response['url'],
    "action_def": action_defs['bring_to_temperature']['url'],
    "source_vessel_template": None,
    "dest_vessel_template": vessel_templates['Outcome vessel']['url'],
    "parent": [dispense_flux_data['url']]
}
heat_to_t_data = client.get_or_create('action-template', data=heat_to_t_data)[0]

{'description': 'Heat to T', 'experiment_template': '9662fef4-7fb0-4a4c-b168-9afb89314693', 'action_def': '199d371e-4772-4a75-8fee-41e8f2eb05a0', 'dest_vessel_template': '4f80aeeb-ee0d-43bf-8451-3b696b1c7946', 'parent': ['http://localhost:8000/api/action-template/b6d539c9-32eb-4fb3-a525-d532a8c0529c/']}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [63]:
dwell_at_t_data = {
    "description": "Dwell at T",
    "source_vessel_decomposable": False,
    "dest_vessel_decomposable": False,
    "experiment_template": template_response['url'],
    "action_def": action_defs['dwell']['url'],
    "source_vessel_template": None,
    "dest_vessel_template": vessel_templates['Outcome vessel']['url'],
    "parent": [heat_to_t_data['url']]
}
dwell_at_t_data = client.get_or_create('action-template', data=dwell_at_t_data)[0]

{'description': 'Dwell at T', 'experiment_template': '9662fef4-7fb0-4a4c-b168-9afb89314693', 'action_def': '56c35b7c-30aa-4270-bcd4-f64929797cf8', 'dest_vessel_template': '4f80aeeb-ee0d-43bf-8451-3b696b1c7946', 'parent': ['http://localhost:8000/api/action-template/48631907-16d1-4bff-97f6-b4dbf83e48da/']}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [64]:
cool_to_t_data = {
    "description": "Cool to T",
    "source_vessel_decomposable": False,
    "dest_vessel_decomposable": False,
    "experiment_template": template_response['url'],
    "action_def": action_defs['cool_to_temperature']['url'],
    "source_vessel_template": None,
    "dest_vessel_template": vessel_templates['Outcome vessel']['url'],
    "parent": [dwell_at_t_data['url']]
}
cool_to_t_data = client.get_or_create('action-template', data=cool_to_t_data)[0]

{'description': 'Cool to T', 'experiment_template': '9662fef4-7fb0-4a4c-b168-9afb89314693', 'action_def': '4998b80c-7f0e-405c-b508-a15e6abbb771', 'dest_vessel_template': '4f80aeeb-ee0d-43bf-8451-3b696b1c7946', 'parent': ['http://localhost:8000/api/action-template/3ae564e5-a060-4649-8097-d9909cf26293/']}
GET: OK. Found 0 results
POST: OK, returning new resource dict
